# Toxicity Classification
This data contains a large number of wikipedia comments that have been classified by human raters for toxic behaviour.
- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate
    
I will be creating a model to predict the probability of each type of toxicity for each comment. 


In [30]:
#import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [17]:
#load dataset
df = pd.read_csv('train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [19]:
import random as rd
df[df.columns[2:]].iloc[rd.randint(0,159570)]

toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
Name: 138341, dtype: int64

In [20]:
df.iloc[rd.randint(0,159570)]['comment_text']

'Your Beautiful wife'

## Preprocess comments for model

#### Tokenising the data
- Coverting each word to a unique identifier so that each word maps to a number

In [21]:
#Text Vectorization layer
from tensorflow.keras.layers import TextVectorization

In [22]:
#split data into features and labels
x = df['comment_text']
y = df[df.columns[2:]].values #putting the dataframe in an array format that can be passed to the model

In [23]:
MAX_FEATURES = 150000 #number of words in the vocab

In [24]:
#initialize the text vectorization layer
#converting words to integers
vectorizer = TextVectorization(max_tokens = MAX_FEATURES,
                               output_sequence_length = 1000,
                              output_mode = 'int')

In [25]:
#train the vectorizer to learn the vocab
vectorizer.adapt(x.values) #converting from series to numpy array

In [26]:
#test vectorizer
vectorizer('Hello world, life is great')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([288, 263, 306,   9, 275], dtype=int64)>

In [27]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [28]:
vectorized_text = vectorizer(x.values)
vectorized_text

<tf.Tensor: shape=(159571, 1000), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [31]:
#create a tensorflow pipeline
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))#parsing through the vectorized test and label
dataset = dataset.cache()#data
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)#prevents bottlenecks

In [74]:
batch_x, batch_y = dataset.as_numpy_iterator().next()

In [34]:
#vectorized samples
batch_x.shape

(16, 1000)

In [35]:
#labels
batch_y.shape

(16, 6)

#### Create the training, validation and test partitions

In [36]:
train = dataset.take(int(len(dataset) * 0.7))
val = dataset.skip(int(len(dataset)* 0.7)).take(int(len(dataset) * 0.2))
test = dataset.skip(int(len(dataset) * 0.9)).take(int(len(dataset) * 0.1))

## Create a Sequential Model

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [39]:
model = Sequential()
#Create the embedding layer
model.add(Embedding(MAX_FEATURES+1, 32))
#implement the Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation = 'tanh')))
#feature extractor, fully connected layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
#final layer
model.add(Dense(6, activation = 'sigmoid'))

In [40]:
model.compile(loss = 'BinaryCrossentropy', optimizer = 'Adam')

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          4800032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [42]:
history = model.fit(train, epochs = 1, validation_data = val)

6981/6981 [==============================] - 14650s 2s/step - loss: 0.0629 - val_loss: 0.0479


In [43]:
history.history

{'loss': [0.06291867047548294], 'val_loss': [0.04789426550269127]}

## Make Predictions
> To make predictions, the text needs to be vectorized

In [49]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [71]:
input_text = vectorizer('You are stupid')

In [72]:
#predicting on a sample text
model.predict(np.expand_dims(input_text,0))

1/1 [==============================] - 0s 180ms/step


array([[0.94944   , 0.08460122, 0.7150964 , 0.03846341, 0.6030794 ,
        0.07969306]], dtype=float32)

In [75]:
test_x, test_y = test.as_numpy_iterator().next()

In [79]:
(model.predict(test_x) > 0.5).astype(int)

1/1 [==============================] - 0s 342ms/step


array([[0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [78]:
test_y

array([[0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

## Evaluate Model

In [80]:
#import metrics
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [82]:
#instantiate the metrics
pre = Precision()
rec = Recall()
acc = CategoricalAccuracy()

In [85]:
#looping through each batch in the test data
for batch in test.as_numpy_iterator():
    #unpack the batch
    xtrue, ytrue = batch
    #make prediction
    yh = model.predict(xtrue)
    
    ytrue = ytrue.flatten() 
    yh = yh.flatten()
    
    pre.update_state(ytrue, yh)
    rec.update_state(ytrue, yh)
    acc.update_state(ytrue, yh) 

1/1 [==============================] - 0s 263ms/step


1/1 [==============================] - 0s 270ms/step


1/1 [==============================] - 0s 475ms/step


1/1 [==============================] - 0s 456ms/step


1/1 [==============================] - 0s 352ms/step


1/1 [==============================] - 0s 484ms/step


1/1 [==============================] - 0s 469ms/step


In [86]:
print(f'Precision:{pre.result().numpy()}, Recall:{rec.result().numpy()}, Accuracy: {acc.result().numpy()}')

Precision:0.8826966285705566, Recall:0.5844001173973083, Accuracy: 0.4854563772678375
